In [28]:
import torch
import torch.nn as nn 
import numpy as np

In [2]:
test = torch.randn((224,224,3))
test.size()

torch.Size([224, 224, 3])

In [76]:
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction_ratio):
        super().__init__()
        self.channels = in_channels
        self.r = reduction_ratio 
        assert self.r > 0
        self.num_features = int(self.channels * self.r)
        self.se = nn.Sequential(nn.Linear(self.channels, self.num_features), 
                                nn.ReLU(), 
                                nn.Linear(self.num_features, self.channels), 
                                nn.Sigmoid())
    
    def forward(self, x):
        x_ = x.mean(dim=(3,2))
        x_ = self.se(x_)
        # channelwise multiplication
        x_ = x_.unsqueeze(-1).unsqueeze(-1)
        x = torch.mul(x, x_)
        return x

In [77]:
a = torch.randn((1,8))
b = torch.randn((1,8,16,16))
a = a.unsqueeze(-1).unsqueeze(-1)
a.size()

torch.Size([1, 8, 1, 1])

In [78]:
c = torch.mul(a,b)
c.size()

torch.Size([1, 8, 16, 16])

In [79]:
test = torch.randn((1, 16, 224,224))
in_channels = 16
se = SEBlock(in_channels, 4)

In [80]:
out = se(test)
out.size()

torch.Size([1, 16, 224, 224])

In [81]:
for p in se.parameters():
    print(p.size())

torch.Size([64, 16])
torch.Size([64])
torch.Size([16, 64])
torch.Size([16])


In [82]:
class MBConv(nn.Module):
    def __init__(self, in_channels, out_channels, filter_size, expand_ratio, stride, se_ratio, padding=1):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.k = filter_size
        self.t = expand_ratio
        self.s = stride
        self.se_ratio = se_ratio
        
        #MBConv1
        middle_dim = self.in_channels * self.t 
        if self.t == 1:
            self.conv = nn.Sequential(nn.Conv2d(middle_dim, middle_dim, 3, self.s, padding, groups=middle_dim, bias=False), 
                                      nn.BatchNorm2d(middle_dim), 
                                      nn.SiLU(inplace=True), 
                                      nn.Conv2d(middle_dim, self.out_channels, 1, bias=False), 
                                      nn.BatchNorm2d(self.out_channels))
        # MBConv6
        else: 
            self.conv = nn.Sequential(nn.Conv2d(self.in_channels, middle_dim, 1, bias=False), 
                                      nn.BatchNorm2d(middle_dim),
                                      nn.SiLU(inplace=True),
                                      nn.Conv2d(middle_dim, middle_dim, 3, self.s, padding, groups=middle_dim, bias=False), 
                                      nn.BatchNorm2d(middle_dim), 
                                      nn.SiLU(inplace=True), 
                                      nn.Conv2d(middle_dim, self.out_channels, 1, bias=False), 
                                      nn.BatchNorm2d(self.out_channels))
        
        self.se = SEBlock(self.out_channels, self.se_ratio)
            
        
    def forward(self, x):
        if self.s == 1 and self.in_channels == self.out_channels: 
            return x + self.se(self.conv(x))
        else:
            return self.se(self.conv(x))

In [83]:
mbconv1 = MBConv(3, 32, 3, 1, 1, 0.25)
mbconv6 = MBConv(3, 32, 3, 6, 1, 0.25)

In [84]:
test = torch.randn((224,224,3))
test = test.unsqueeze(0)
test = test.transpose(1,3)
test.size()

torch.Size([1, 3, 224, 224])

In [85]:
out1 = mbconv1(test)
out1.size()

torch.Size([1, 32, 224, 224])

In [86]:
test = torch.randn((224,224,3))
test = test.unsqueeze(0)
test = test.transpose(1,3)
test.size()

torch.Size([1, 3, 224, 224])

In [87]:
out2 = mbconv6(test)
out2.size()

torch.Size([1, 32, 224, 224])

In [88]:
class EfficientNet(nn.Module):
    def __init__(self, config, name='efficientnet-b0'):
        super().__init__()
        self.config = config[name]
        self.num_classes = self.config['num_classes']
        self.dropout = self.config['dropout']
        self.in_channels = self.config['in_channels']
        self.out_channels = self.config['out_channels']
        self.stages_config = self.config['stages']
        
        self.stage1 = nn.Sequential(nn.Conv2d(self.in_channels, 32, 3, stride=2, padding=1), 
                                    nn.BatchNorm2d(32), 
                                    nn.SiLU(inplace=True))
        self.modules = [_create_stage(params) for params in self.stages_config]
        self.stages = nn.Sequential(*self.modules)
        self.final = nn.Sequential(nn.Conv2d(320, self.out_channels, 1, padding='same'), 
                                   nn.BatchNorm2d(self.out_channels), 
                                   nn.AdaptiveAvgPool2d(1), 
                                   nn.Flatten(),
                                   nn.Dropout(self.dropout), 
                                   nn.Linear(self.out_channels, self.num_classes))
        
        
    def forward(self, x):
        x = self.stage1(x)
        x = self.stages(x)
        x = self.final(x)
        return x
    
    def _create_stage(self, params):
        expand_ratio, filter_size, num_repeats, in_channels, out_channels, stride, padding, se_ratio = params
        modules = []
        for i in range(num_repeats):
            if i == (num_repeats-1):
                modules.append(MBConv(in_channels, out_channels, filter_size, expand_ratio, stride, se_ratio, 1))
            else:
                modules.append(MBConv(in_channels, in_channels, filter_size, expand_ratio, 1, se_ratio, padding))

        return modules

In [89]:
configs = {
    'efficientnet-b0': {
        'num_classes': 1000,
        'in_channels': 3,
        'out_channels': 1280,
        'dropout': 0.2,
        'stages': [
            # expand_ratio, filter_size, num_repeats, in_channels, out_channels, stride, padding, se_ratio
            (1, 3, 1, 32, 16, 1, 'same', 0.25),
            (6, 3, 2, 16, 24, 2, 1, 0.25),
            (6, 5, 2, 24, 40, 2, 1, 0.25),
            (6, 3, 3, 40, 80, 2, 1, 0.25),
            (6, 5, 3, 80, 112, 1, 'same', 0.25),
            (6, 5, 4, 112, 192, 2, 1, 0.25),
            (6, 3, 1, 192, 320, 1, 'same', 0.25)
        ]
                
    }
}

In [90]:
net = EfficientNet(configs)
net

EfficientNet(
  (stage1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU(inplace=True)
  )
  (stages): Sequential(
    (0): Sequential(
      (0): MBConv(
        (conv): Sequential(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): SiLU(inplace=True)
          (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (se): SEBlock(
          (se): Sequential(
            (0): Linear(in_features=16, out_features=4, bias=True)
            (1): ReLU()
            (2): Linear(in_features=4, out_features=16, bias=True)
            (3): Sigmoid()
          )
        )
  

In [91]:
test = torch.randn(1, 3, 333, 333)
test.size()

torch.Size([1, 3, 333, 333])

In [92]:
out = net(test)
out.size()

torch.Size([1, 1000])

In [29]:
in_height, in_width = 224,224
filter_height, filter_width = 3,3
strides=(None,2,2)
out_height = np.ceil(float(in_height) / float(strides[1]))
out_width  = np.ceil(float(in_width) / float(strides[2]))

#The total padding applied along the height and width is computed as:

if (in_height % strides[1] == 0):
    pad_along_height = max(filter_height - strides[1], 0)
else:
    pad_along_height = max(filter_height - (in_height % strides[1]), 0)
if (in_width % strides[2] == 0):
    pad_along_width = max(filter_width - strides[2], 0)
else:
    pad_along_width = max(filter_width - (in_width % strides[2]), 0)

print(pad_along_height, pad_along_width)
  
#Finally, the padding on the top, bottom, left and right are:

pad_top = pad_along_height // 2
pad_bottom = pad_along_height - pad_top
pad_left = pad_along_width // 2
pad_right = pad_along_width - pad_left

print(pad_left, pad_right, pad_top, pad_bottom)

1 1
0 1 0 1


In [23]:
test = torch.randn(1,3,224,224)
test.size()

torch.Size([1, 3, 224, 224])

In [26]:
same = nn.Conv2d(3, 16, 3, 2, 1, bias=False)
out = same(test)
out.size()

torch.Size([1, 16, 112, 112])

In [30]:
3//2

1